In [ ]:
from sportsipy.nhl.teams import Teams
from scipy.stats import pearsonr
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import statistics
from matplotlib import pyplot as plt

In [2]:
print('Correlation(WinRatio,GoalsForRatio)\n')

maxCorr= -1.1
minCorr= 1.1
minYear = 2021
maxYear = 2021
corrList = []
# https://towardsdatascience.com/web-scraping-nba-stats-4b4f8c525994
def getGa(abbr,year):
    url = 'https://www.hockey-reference.com/teams/' + abbr + '/' + str(year) + '.html'
    html = urlopen(url)
    soup = BeautifulSoup(html,features="lxml")
    
    # avoid the first header row
    rows = soup.findAll('tr')[1:]
    all_stats = [[td.getText() for td in rows[i].findAll('td')]for i in range(len(rows))]
    all_stats2 = [e for e in all_stats if e]
    all_stats3=[x for x in all_stats2 if len(x)>3]
    team_stats = all_stats3[0]
    if year > 2005:
        goals_against = team_stats[8]
    else:
        goals_against = team_stats[9]
    return(goals_against)

corrd = {}
for year in range(2006, 2021):
    if year == 2005:
        continue
    teams = list(Teams(year))
    statd = {}
    for i,team in enumerate(teams[0:len(teams)]):
        G = team.games_played
        WinPct = round(100*(team.wins+(.5*team.overtime_losses))/G,3)
        GF = team.goals_for
        GA = int(getGa(team.abbreviation,year))
        statd[i] = {'Team' : team.name,
                    'W' : team.wins,
                    'L' : team.losses,
                    'OTL' : team.overtime_losses,                
                    'G' : G,
                    'WinPct' : WinPct,
                    'GF' : GF,
                    'GA' : GA,
                    'GD' : GF-GA,
                    'GfrPct' : round(100*GF/(GF+GA),3),
                    'GD_per' : round((GF-GA)/G,2)}
                    
    df = pd.DataFrame(statd)
    df = df.transpose()

    corr, _ = pearsonr(df['GfrPct'].tolist(), df['WinPct'].tolist())
    corrList.append(corr)
    
    if maxCorr < minCorr:
        minCorr = corr
        maxCorr = corr
        minYear = year
        maxYear = year
        print('%s: %s (new min) (new max)' % (year,round(corr,3)))
    elif corr < minCorr:
        minCorr= corr
        minYear = year
        print('%s: %s (new min)' % (year,round(corr,3)))
    elif corr > maxCorr:
        maxCorr= corr
        maxYear = year
        print('%s: %s (new max)' % (year,round(corr,3)))
    else: 
        print('%s: %s' % (year,round(corr,3)))

print('MIN:%s (%s)' % (minYear,round(minCorr,3)))
print('MAX:%s (%s)' % (maxYear,round(maxCorr,3)))
print('AVG:%s' % (round(sum(corrList)/len(corrList),3)))

Correlation(WinRatio,GoalsForRatio)

2006: 0.928 (new min) (new max)
2007: 0.943 (new max)
2008: 0.917 (new min)
2009: 0.951 (new max)
2010: 0.946
2011: 0.925
2012: 0.9 (new min)
2013: 0.923
2014: 0.968 (new max)
2015: 0.961
2016: 0.951
2017: 0.974 (new max)
2018: 0.981 (new max)
2019: 0.968
2020: 0.968
MIN:2012 (0.9)
MAX:2018 (0.981)
AVG:0.947


In [ ]:
'''
MIN:2012 (0.9)
MAX:2018 (0.981)
AVG:0.947
'''

In [47]:
year=2012
teams = list(Teams(year))
statd = {}

for i,team in enumerate(teams[0:len(teams)]):
    G = team.games_played
    WinPct = round(100*(team.wins+(.5*team.overtime_losses))/G,3)
    GF = team.goals_for
    GA = int(getGa(team.abbreviation,year))
    statd[i] = {'Team' : team.name,
                'W' : team.wins,
                'L' : team.losses,
                'OTL' : team.overtime_losses,                
                'G' : G,
                'WinPct' : WinPct,
                'GF' : GF,
                'GA' : GA,
                'GD' : GF-GA,
                'GD_per' : round((GF-GA)/G,2)} 
        
df = pd.DataFrame(statd)
df = df.transpose()

x = list(df['WinPct'])
seq = sorted(x,reverse=True)
index = [seq.index(v) for v in x]
df['Iwin'] =  index

x = list(df['GfrPct'])
seq = sorted(x,reverse=True)
index = [seq.index(v) for v in x]
df['Igfp'] =  index

df['Delta'] = df['Igfp']-df['Iwin']
x = list(df['Delta'])
seq = sorted(x,reverse=True)
index = [seq.index(v) for v in x]
df['Idel'] =  index

df = df[['Team','W','L','G','GF','GA','WinPct','GD','GD_per','GfrPct','Iwin','Igfp','Delta','Idel']]
df = df.sort_values(['WinPct'],ascending = (False))
df['WINvs50'] = df['WinPct']-50
df_mdian = df.loc[abs(df['GD']) == min(abs((df['GD'])))]
df_mdian = df_mdian[abs(df_mdian['WINvs50']) == min(abs(df_mdian['WINvs50']))].head(1)
df_lucky = df[df['Delta'] == max(df['Delta'])].head(1)
df_nluck = df[df['Delta'] == min(df['Delta'])].head(1)
print('%s Luckiest,Median,Unluckiest Teams:' % year)
df_ext = pd.concat([df_lucky,pd.concat([df_mdian,df_nluck],sort=False)],sort=False)
print(df_ext.to_string(index=False))

2012 Luckiest,Median,Unluckiest Teams:
               Team   W OTL   L   G   GF   GA   GD  WinPct GD_per  Iwin  Igd  Delta  Idel WINvs50
   Florida Panthers  38  18  26  82  197  216  -19  57.317  -0.23    13   19      6     0   7.317
 Montreal Canadiens  31  16  35  82  207  214   -7  47.561  -0.09    27   14    -13    29  -2.439
 Montreal Canadiens  31  16  35  82  207  214   -7  47.561  -0.09    27   14    -13    29  -2.439


In [ ]:
'''
2012 Luckiest,Median,Unluckiest Teams: (most anomolous)
               Team   W OTL   L   G   GF   GA   GD  WinPct GD_per  Iwin  Igd  Delta  Idel WINvs50
   Florida Panthers  38  18  26  82  197  216  -19  57.317  -0.23    13   19      6     0   7.317
 Montreal Canadiens  31  16  35  82  207  214   -7  47.561  -0.09    27   14    -13    29  -2.439
 Montreal Canadiens  31  16  35  82  207  214   -7  47.561  -0.09    27   14    -13    29  -2.439

2018 Luckiest,Median,Unluckiest Teams: (most predictable)
                Team   W OTL   L   G   GF   GA  GD  WinPct GD_per  Iwin  Igd  Delta  Idel WINvs50
 Washington Capitals  49   7  26  82  256  238  18  64.024   0.22     5   12      7     0  14.024
     St. Louis Blues  44   6  32  82  223  222   1  57.317   0.01    17   18      1     5   7.317
   Los Angeles Kings  45   8  29  82  237  202  35  59.756   0.43    11    6     -5    28   9.756
'''

In [3]:
year=2012
teams = list(Teams(year))
statd = {}

for i,team in enumerate(teams[0:len(teams)]):
    G = team.games_played
    WinPct = round(100*(team.wins+(.5*team.overtime_losses))/G,3)
    GF = team.goals_for
    GA = int(getGa(team.abbreviation,year))
    statd[i] = {'Team' : team.name,
                'W' : team.wins,
                'L' : team.losses,
                'OTL' : team.overtime_losses,                
                'G' : G,
                'WinPct' : WinPct,
                'GF' : GF,
                'GA' : GA,
                'GD' : GF-GA,
                'GD_per' : round((GF-GA)/G,2)} 
        
df = pd.DataFrame(statd)
df = df.transpose()

x = list(df['WinPct'])
seq = sorted(x,reverse=True)
index = [seq.index(v) for v in x]
df['Iwin'] =  index

x = list(df['GD'])
seq = sorted(x,reverse=True)
index = [seq.index(v) for v in x]
df['Igd'] =  index

df['Delta'] = df['Igfp']-df['Iwin']
x = list(df['Delta'])
seq = sorted(x,reverse=True)
index = [seq.index(v) for v in x]
df['Idel'] =  index

df = df[['Team','W','L','G','GF','GA','WinPct','GD_per','GfrPct','Iwin','Igfp','Delta','Idel']]
df = df.sort_values(['WinPct'],ascending = (False))
print('%s NHL Season - Ranked by %% Games Won [>LosPct]' % year)
with pd.option_context('display.max_rows',6):
    display(df)

df = df.sort_values('GfrPct',ascending=False)
print('\n%s NHL Season - Ranked by %% Points Scored [<GfrPct]' % year)
with pd.option_context('display.max_rows',6):
    display(df)

df = df.sort_values('Idel',ascending=True)
print('\n%s NHLb Season - Ranked by Luck [<Delta (Igfp-Iwin)]' % year)
with pd.option_context('display.max_rows',6):
    display(df)

KeyError: 'Igfp'

In [ ]:
minDelta = 0
maxDelta = 0
deltaList = []
for year in range(2006, 2021):
    if year == 2005:
        continue
    teams = list(Teams(year))
    statd = {}
    for i,team in enumerate(teams[0:len(teams)]):
        G = team.games_played
        WinPct = round(100*(team.wins+(.5*team.overtime_losses))/G,3)
        GF = team.goals_for
        GA = int(getGa(team.abbreviation,year))
        statd[i] = {'Team' : team.name,
                    'WinPct' : WinPct,
                    'GfrPct' : round(100*GF/(GF+GA),3)}
        
    df = pd.DataFrame(statd)
    df = df.transpose()

    x = list(df['WinPct'])
    seq = sorted(x,reverse=True)
    index = [seq.index(v) for v in x]
    df['Iwin'] =  index

    x = list(df['GfrPct'])
    seq = sorted(x,reverse=True)
    index = [seq.index(v) for v in x]
    df['Igfp'] =  index

    df['Delta'] = df['Igfp']-df['Iwin']
    x = list(df['Delta'])
    
    locMinDelta = int(min(df['Delta']))
    locMinTeam = df.Team[df['Delta'] == locMinDelta]
    locMinTeam = locMinTeam.to_string(index=False)
    locMaxDelta = int(max(df['Delta']))
    locMaxTeam = df.Team[df['Delta'] == locMaxDelta]
    locMaxTeam = locMaxTeam.to_string(index=False)
    
    if maxDelta == minDelta:
        minDelta = locMinDelta
        maxDelta = locMaxDelta
        minTeam = locMinTeam
        maxTeam = locMaxTeam
        minYear = year
        maxYear = year
        print('%s %s: %s (new minDelta) (new maxDelta)' % (year,locMinTeam,round(minDelta,3)))
    elif locMinDelta < minDelta:
        minDelta= locMinDelta
        minTeam = locMinTeam
        minYear = year
        print('%s %s: %s (new min)' % (year,locMinTeam,round(minDelta,3)))
    elif locMaxDelta > maxDelta:
        maxDelta = locMaxDelta
        maxTeam = locMaxTeam
        maxYear = year
        print('%s %s: %s (new max)' % (year,locMaxTeam,round(maxDelta,3)))
    else: 
        pass
    
    for d in x:
        deltaList.append(d)

deltaList=sorted(deltaList,reverse=False)

In [ ]:
print('median: ' + str(statistics.median(deltaList)))
print('mean: ' + str(round(statistics.mean(deltaList),2)))
print('mode: ' + str(statistics.mode(deltaList)))
print('pstdev: ' + str(round(statistics.pstdev(deltaList),2)))
print('pvariance: ' + str(round(statistics.pvariance(deltaList),2)))

quartiles=[round(q, 1) for q in statistics.quantiles(deltaList, n=4)]
deciles=[round(q, 1) for q in statistics.quantiles(deltaList, n=10)]
print('quartiles: '+ str(quartiles))
print('deciles: '+ str(deciles))

buck = len(set(deltaList))
plt.hist(deltaList, buck)
plt.show()